In [2]:
from rdflib import Graph, URIRef, Namespace
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from tqdm import tqdm
import pandas as pd

In [4]:
#rdflib_dbpedia_graph = Graph("SPARQLStore")
#rdflib_dbpedia_graph.open("http://localhost:9999/bigdata/sparql")

graph = Graph()
graph.parse("")

<Graph identifier=N71274ffeeb0d4e0e89cfdb1605e1d63a (<class 'rdflib.graph.Graph'>)>

In [5]:
query = """
SELECT DISTINCT ?p
WHERE {
  ?s ?p ?o.
}"""
pred_res_list = graph.query(query)

In [6]:
count_total_query = """
SELECT (COUNT(*) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

count_total_obj_query = """
SELECT (COUNT(DISTINCT ?o) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

inv_func_dict = dict()
for pred_res in tqdm(pred_res_list):
    query_pred = pred_res[0]

    try:
        replaced_count_total_query = count_total_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_query)
        total_count = 0
        for res in count_res:
            total_count = res.totalCount

        replaced_count_total_obj_query = count_total_obj_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_obj_query)
        total_obj = 0
        for res in count_res:
            total_obj = res.totalCount
        
        inv_func_dict[str(query_pred)] = float(total_obj)/float(total_count)
    except Exception as e:
        print(e)
        print("exception raised: " + str(query_pred))

100%|█████████████████████████████████████████| 715/715 [01:58<00:00,  6.02it/s]


In [7]:
inv_func_list = list()
for key, value in inv_func_dict.items():
    inv_func_list.append({
        'predicate': key,
        'inverse_functionality': value
    })

In [8]:
inv_func_df = pd.DataFrame(inv_func_list)

In [9]:
inv_func_df

,predicate,inverse_functionality
0,http://dbkwik.webdatacommons.org/ontology/wiki...,0.062096
1,http://dbkwik.webdatacommons.org/starwars.wiki...,0.069264
2,http://www.w3.org/2004/02/skos/core#altLabel,1.000000
3,http://dbkwik.webdatacommons.org/starwars.wiki...,0.281726
4,http://purl.org/dc/terms/subject,0.036142
...,...,...
710,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
711,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
712,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
713,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000


In [10]:
inv_func_df[inv_func_df['inverse_functionality'] > 0.5]

,predicate,inverse_functionality
2,http://www.w3.org/2004/02/skos/core#altLabel,1.000000
5,http://dbkwik.webdatacommons.org/ontology/wiki...,0.746123
6,http://xmlns.com/foaf/0.1/thumbnail,1.000000
10,http://dbkwik.webdatacommons.org/ontology/wiki...,0.954772
11,http://www.w3.org/2000/01/rdf-schema#label,0.992669
...,...,...
710,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
711,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
712,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
713,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000


In [11]:
query = """
SELECT DISTINCT ?p
WHERE {
  ?s ?p ?o.
}"""
pred_res_list = graph.query(query)

In [12]:
count_total_query = """
SELECT (COUNT(*) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

count_total_subj_query = """
SELECT (COUNT(DISTINCT ?s) AS ?totalCount)
WHERE {
    ?s <:pred> ?o
}
"""

func_dict = dict()
for pred_res in tqdm(pred_res_list):
    query_pred = pred_res[0]

    try:
        replaced_count_total_query = count_total_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_query)
        total_count = 0
        for res in count_res:
            total_count = res.totalCount

        replaced_count_total_subj_query = count_total_subj_query.replace(':pred', str(query_pred))
        count_res = graph.query(replaced_count_total_subj_query)
        total_obj = 0
        for res in count_res:
            total_obj = res.totalCount
        
        func_dict[str(query_pred)] = float(total_obj)/float(total_count)
    except Exception as e:
        print(e)
        print("exception raised: " + str(query_pred))

100%|█████████████████████████████████████████| 715/715 [01:58<00:00,  6.05it/s]


In [13]:
func_list = list()
for key, value in func_dict.items():
    func_list.append({
        'predicate': key,
        'functionality': value
    })

In [14]:
func_df = pd.DataFrame(func_list)

In [15]:
func_df

,predicate,functionality
0,http://dbkwik.webdatacommons.org/ontology/wiki...,0.035206
1,http://dbkwik.webdatacommons.org/starwars.wiki...,0.977656
2,http://www.w3.org/2004/02/skos/core#altLabel,0.720581
3,http://dbkwik.webdatacommons.org/starwars.wiki...,0.098430
4,http://purl.org/dc/terms/subject,0.485668
...,...,...
710,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
711,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
712,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
713,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000


In [16]:
func_df[func_df['functionality'] > 0.5]

,predicate,functionality
1,http://dbkwik.webdatacommons.org/starwars.wiki...,0.977656
2,http://www.w3.org/2004/02/skos/core#altLabel,0.720581
6,http://xmlns.com/foaf/0.1/thumbnail,1.000000
7,http://xmlns.com/foaf/0.1/depiction,0.610629
11,http://www.w3.org/2000/01/rdf-schema#label,0.999988
...,...,...
710,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
711,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
712,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000
713,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000


In [17]:
inversability_df = pd.merge(func_df, inv_func_df, on='predicate')
inversability_df['inversability'] = inversability_df['inverse_functionality'] * inversability_df['functionality']


In [18]:
inversability_df

,predicate,functionality,inverse_functionality,inversability
0,http://dbkwik.webdatacommons.org/ontology/wiki...,0.035206,0.062096,0.002186
1,http://dbkwik.webdatacommons.org/starwars.wiki...,0.977656,0.069264,0.067716
2,http://www.w3.org/2004/02/skos/core#altLabel,0.720581,1.000000,0.720581
3,http://dbkwik.webdatacommons.org/starwars.wiki...,0.098430,0.281726,0.027730
4,http://purl.org/dc/terms/subject,0.485668,0.036142,0.017553
...,...,...,...,...
710,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000
711,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000
712,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000
713,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000


In [19]:
inversability_df[inversability_df['inverse_functionality'] > 0.5]

,predicate,functionality,inverse_functionality,inversability
2,http://www.w3.org/2004/02/skos/core#altLabel,0.720581,1.000000,0.720581
5,http://dbkwik.webdatacommons.org/ontology/wiki...,0.466285,0.746123,0.347906
6,http://xmlns.com/foaf/0.1/thumbnail,1.000000,1.000000,1.000000
10,http://dbkwik.webdatacommons.org/ontology/wiki...,0.194282,0.954772,0.185495
11,http://www.w3.org/2000/01/rdf-schema#label,0.999988,0.992669,0.992658
...,...,...,...,...
710,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000
711,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000
712,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000
713,http://dbkwik.webdatacommons.org/starwars.wiki...,1.000000,1.000000,1.000000


In [20]:
inversability_df.to_csv('')